使用MinHashLSHForest对微博新闻句子进行检索; 针对某句话进行Query，查找Top-3相似的句子

### 思路
1. 数据加载
2. 对数据进行整理
- 切分句子
- 对每个句子进行分词
3. 创建minHash, 建立minHash的语料库forest(设立index)
4. 进行检索
- 对query进行分词并创建minHash
- 在forest中查询, 输出jaccard值和序号
- 得到jaccard值最高的前三个句子

In [210]:
## 数据加载
import re
weibo = open(r'E:\bi_course\L6\sat_course\课堂资料\L6-code\L6\weibos.txt', encoding='UTF-8')
text = weibo.read()

# 查看数据后, 以句号，叹号，问号作为句子分隔; 并去掉\n和\u200b符号
sentences = re.split('[#。！？]', text.replace('\n', '').replace('\u200b','').replace('“','').replace('”','').replace(' ',''))

# 删除每一个空行
for i in range(len(sentences))[::-1]: #删除元素之后list后面元素向前移动，而且长度变短, 倒叙删除
    if sentences[i] == '':  #删除为空的句子
        del sentences[i]
    elif sentences[i][1] == '，': #删除以","开头的句子
        sentences[i].replace('，','')

print('句子总数:',len(sentences))
print(sentences)

句子总数: 39
[&#39;斯科拉里愿意执教国足&#39;, &#39;上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用&#39;, &#39;国足输给叙利亚之后，里皮辞职&#39;, &#39;谁将成为新主帅，成为广大球迷关注的焦点&#39;, &#39;目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里&#39;, &#39;据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决&#39;, &#39;据了解&#39;, &#39;比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战&#39;, &#39;辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利&#39;, &#39;这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话&#39;, &#39;至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商&#39;, &#39;让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……&#39;, &#39;来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足&#39;, &#39;老头有点意思，凡是里皮干过的地方，他就想试试&#39;, &#39;当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛&#39;, &#39;可以试试&#39;, &#39;斯科拉里的水平，还不如里皮&#39;, &#39;斯科拉里，看好的不是国足，而是年薪……非常应该辞职&#39;, &#39;中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有&#39;, &#39;从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了&#39;, &#39;穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光&#39;, &#39;主教练说：赛前老子就再三交代，这一场无论如何都不能赢中国队&#39;, &#39;中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日

In [214]:
# 设置停用词

stopwords = open(r"E:\bi_course\L3\tue_course\chinese_stopwords.txt","r", encoding = 'utf-8')
stop = [ word for  word in stopwords.readlines()] #[:-1]表示删除\n字符

In [215]:
#分词函数

from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg

def get_item(item_text):
    item_str = ""
    item = pseg.cut(item_text) #利用jieba对句子进行分词
    for  i in list(item):
        #去掉停用词
         if i.word not in list(stop):  
            item_str += i.word

            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "

    return item_str

# 对item_str创建MinHash函数
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8')) 
    return temp

In [216]:
# 得到分词后的documents

documents = []

for item_text in sentences:
    # 将words进行分词
    item_str = get_item(item_text)
    documents.append(item_str)

# documents

In [217]:
# 创建LSH Forest及MinHash对象

from datasketch import MinHash, MinHashLSH, MinHashLSHForest

minhash_ls = []
forest = MinHashLSHForest()

for i in range(len(documents)):

    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_ls.append(temp)
    forest.add(i, temp)

# index所有key，以便可以进行检索
forest.index()


In [218]:
query = '爱护生命，远离男足'
# 将item_text进行分词
item_str = get_item(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与query相似的Top-K个邻居
result = forest.query(minhash_query, 3)

for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_ls[result[i]]), documents[result[i]].replace(' ', ''))

print("Top 3相似句子", result)

35 0.234375 国足输给叙利亚后，里皮坐不住了，直接辞职了难怪有网友说，爱护生命，远离男足
36 0.2109375 男足的水平也就跟南极洲企鹅踢球
23 0.078125 ，球员委屈的说：七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛
Top 3相似句子 [35, 36, 23]
